In [1]:
from pitchtypes import Pitch, SpelledPitch, EnharmonicPitch, LogFreqPitch
from math import log

## Basic Pitch Types

Both _SpelledPitch_ and _EnharmonicPitch_ can be initialised from commonly used pitch notation. However, _SpelledPitch_ will (you guessed it) retain the spelling while _EnharmonicPitch_ converts to the corresponding enharmonically equivalent pitch (internally stored as the corresponding integer in MIDI representation). When printing 'black keys', _EnharmonicPitch_ will need to add an accidental, which by default is a sharp (the 'flat' version can be obtained by explicitly calling the _name_ function with the corresponding argument). _SpelledPitch_ can be converted to _EnharmonicPitch_ but not the other way around.

In [2]:
for pitch in ["F##", "Bbb", "C#", "Db"]:
    print(f"SpelledPitch('{pitch}'):\t{SpelledPitch(pitch)}")
    p = EnharmonicPitch(pitch)
    print(f"EnharmonicPitch('{pitch}'):\t{p}\t[{p.name(sharp_flat='sharp')}/{p.name(sharp_flat='flat')}]")
    print(f"{SpelledPitch(pitch)} --> {EnharmonicPitch(SpelledPitch(pitch))}")
    # print(f"{EnharmonicPitch(pitch)} --> {SpelledPitch(EnharmonicPitch(pitch))}")  # NotImplementedError
    print()

SpelledPitch('F##'):	F##
EnharmonicPitch('F##'):	G	[G/G]
F## --> G

SpelledPitch('Bbb'):	Bbb
EnharmonicPitch('Bbb'):	A	[A/A]
Bbb --> A

SpelledPitch('C#'):	C#
EnharmonicPitch('C#'):	C#	[C#/Db]
C# --> C#

SpelledPitch('Db'):	Db
EnharmonicPitch('Db'):	C#	[C#/Db]
Db --> C#



### Pitch versus Pitch Classe

In fact, the example above is using pitch _classes_, that is the octave is ignored. If you want actual pitches, you will need to specify the octave.

In [3]:
for pitch in ["Dbb1", "C2", "B#3", "C4"]:
    midi = int(EnharmonicPitch(pitch))
    print(f"{SpelledPitch(pitch)}\t{EnharmonicPitch(pitch)}\t(MIDI: {midi} <--> {EnharmonicPitch(midi)})")

Dbb1	C1	(MIDI: 24 <--> C1)
C2	C2	(MIDI: 36 <--> C2)
B#3	C4	(MIDI: 60 <--> C4)
C4	C4	(MIDI: 60 <--> C4)


As you can see, _EnharmonicPitch_ can also be converted back and forth from/to the corresponding MIDI integer representation. And if you have a pitch object, you can easily convert it to the corresponding pitch _class_:

In [4]:
p = EnharmonicPitch("C4")
pc = p.to_pitch_class()
print(f"{p} --> {pc}")

C4 --> C


### Frequencies

If you are interested in the frequency corresponding to a specific pitch...well, you should think twice what that means, but here are some ways to deal with frequencies.

We use the _LogFreqPitch_ type to represent frequencies. As the name suggests, it corresponds to the logarithm of the frequency (in Hertz) and is internally represented as a float:

In [5]:
print(LogFreqPitch(log(440.1)))
print(LogFreqPitch(440.1, is_freq=True))
print(f"{float(LogFreqPitch(log(440.1)))} == {log(440.1)}")

440.1Hz
440.1Hz
6.0870019738170456 == 6.0870019738170456


For other pitch types, it is not obvious how they relate to frequency, because this generally depends on the specific tuning being used. To still be as user-friendly as possible, we assume that people using _EnharmonicPitch_ will generally use twelve-tone equal temperament, so we provide a default conversion to frequency:

In [6]:
print(EnharmonicPitch("A4").freq())
print(LogFreqPitch(EnharmonicPitch("A4")))

440.0
440.Hz


In contrast, _SpelledPitch_ pitch contains additional information that may be used to adjust the frequency in an appropriate way, depending on the tuning and/or the musical context. So we do not provide a default conversion and the following would result in an _NotImplementedError_:

In [7]:
# LogFreqPitch(SpelledPitch("A4"))  # NotImplementedError

You can still perform the conversion explicitly, in which case we trust you that you know what you are doing:

In [8]:
LogFreqPitch(EnharmonicPitch(SpelledPitch("A4")))

440.Hz

## Intervals and Arithmetic

ToDo...

In [9]:
p1 = SpelledPitch("C#4")
p2 = SpelledPitch("Gb5")
i = p2 - p1
print(f"{p2} - {p1} = {i}")
print(f"{p2} + {i} = {p2 + i}")
print(f"{p2} - {i} = {p2 - i}")
print(f"{i} - {i} = {i - i}")
print(f"{i} + {i} = {i + i}")
# p1 + p2                                   # TypeError (cannot add pitches)
# SpelledPitch("G") - SpelledPitch("G4")    # TypeError (cannot mix pitches with pitch classes)
# SpelledPitch("G") - EnharmonicPitch("C")  # TypeError (cannot mix different pitch types)

Gb5 - C#4 = dd5+1
Gb5 + dd5+1 = Dbbb6
Gb5 - dd5+1 = C#4
dd5+1 - dd5+1 = P1+0
dd5+1 + dd5+1 = dddd2+2


## Converters

When implementing new pitch types, you can register converters to allow (implicit) conversion to and from existing pitch types.

In [10]:
class PitchA(Pitch): pass
class PitchB(Pitch): pass
class PitchC(Pitch): pass

Pitch.register_converter(from_type=PitchA, 
                         to_type=PitchB, 
                         conv_func=lambda pitch_a: PitchB(pitch_a._value))
Pitch.register_converter(from_type=PitchB, 
                         to_type=PitchC, 
                         conv_func=lambda pitch_b: PitchC(pitch_b._value))

print(PitchA("foo").convert_to(PitchB))
print(PitchB("bar").convert_to(PitchC))

PitchB(foo)
PitchC(bar)


### Implicit Converters

In [11]:
print(PitchA("baz").convert_to(PitchC))  # What?!

PitchC(baz)


We did not provide a converter from PitchA to PitchC, instead, an _implicit_ converter was automatically added by chaining the other two: PitchA --> PitchB --> PitchC. This can be supressed as follows (here for the inverse conversion):

In [12]:
Pitch.register_converter(from_type=PitchC,
                         to_type=PitchB, 
                         conv_func=lambda pitch_c: PitchB(pitch_c._value))
Pitch.register_converter(from_type=PitchB,
                         to_type=PitchA, 
                         conv_func=lambda pitch_b: PitchA(pitch_b._value),
                         create_implicit_converters=False)  # Here, the implicit converter would have been created!
print(PitchC("foo").convert_to(PitchB))
print(PitchB("bar").convert_to(PitchA))
# print(PitchC("baz").convert_to(PitchA))  # NotImplementedError

PitchB(foo)
PitchA(bar)


You can look at all registered converters and see how the implicit converter from above is made up from two separate conversion steps arranged into a pipeline. In fact, there are only the two predefined converters we already know (_SpelledPitch_ --> _EnharmonicPitch_, _EnharmonicPitch_ --> _LogFreqPitch_) and those we just defined (including the implicitly defined one):

In [13]:
for from_type, d in Pitch._converters.items():
    print(f"from: {from_type}")
    for to_type, f in d.items():
        print(f"    to: {to_type}:\t conversion pipeline: {f}")

from: <class 'pitchtypes.datatypes.SpelledPitch'>
    to: <class 'pitchtypes.datatypes.EnharmonicPitch'>:	 conversion pipeline: [<function EnharmonicPitch.convert_from_SpelledPitch at 0x1094d2670>]
from: <class 'pitchtypes.datatypes.EnharmonicPitch'>
    to: <class 'pitchtypes.datatypes.LogFreqPitch'>:	 conversion pipeline: [<function LogFreqPitch.convert_from_midi_pitch at 0x1094d51f0>]
from: <class '__main__.PitchA'>
    to: <class '__main__.PitchB'>:	 conversion pipeline: [<function <lambda> at 0x1094d5c10>]
    to: <class '__main__.PitchC'>:	 conversion pipeline: [<function <lambda> at 0x1094d5c10>, <function <lambda> at 0x1094d5b80>]
from: <class '__main__.PitchB'>
    to: <class '__main__.PitchC'>:	 conversion pipeline: [<function <lambda> at 0x1094d5b80>]
    to: <class '__main__.PitchA'>:	 conversion pipeline: [<function <lambda> at 0x1094d5940>]
from: <class '__main__.PitchC'>
    to: <class '__main__.PitchB'>:	 conversion pipeline: [<function <lambda> at 0x1094d58b0>]
